# Pytorch GPU support

The Palmetto cluster has [many GPU compute nodes](https://docs.rcd.clemson.edu/palmetto/compute/hardware#infiniband-phases). A crucial feature of PyTorch is the support of GPUs--short for Graphics Processing Unit. A GPU can perform many thousands of small operations in parallel, making it perfect for performing large matrix operations in neural networks. _You do not need to know much about GPU programming to use PyTorch on the GPU!_

When comparing GPUs to CPUs, we can list the following main differences (credit: [Kevin Krewell, 2009](https://blogs.nvidia.com/blog/2009/12/16/whats-the-difference-between-a-cpu-and-a-gpu/)) 

<left style="width: 100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial2/comparison_CPU_GPU.png?raw=1" width="700px"></left>

CPUs and GPUs have both different advantages and disadvantages, which is why many computers contain both components and use them for different tasks. In case you are not familiar with GPUs, you can read up more details in this [NVIDIA blog post](https://blogs.nvidia.com/blog/2009/12/16/whats-the-difference-between-a-cpu-and-a-gpu/) or [here](https://www.intel.com/content/www/us/en/products/docs/processors/what-is-a-gpu.html). 



GPUs can accelerate the training of your network up to a factor of $100$ which is essential for large neural networks. PyTorch implements a lot of functionality for supporting GPUs (mostly those of NVIDIA due to the libraries [CUDA](https://developer.nvidia.com/cuda-zone) and [cuDNN](https://developer.nvidia.com/cudnn)). First, let's check whether you have a GPU available:

In [ ]:
import torch

# And our input function
from utils import create_answer_box

In [ ]:
create_answer_box("Have you used GPUs for computing before? If so, what for?", "02-01")

In [ ]:
gpu_avail = torch.cuda.is_available()
print(f'Is the GPU available? {"Yes" if gpu_avail else "No"}')
assert gpu_avail, 'No graphics card available!'

You can information about your GPU usage by opening a terminal and running the `nvidia-smi` command.

In [ ]:
!nvidia-smi

By default, all tensors you create are stored on the CPU. We can push a tensor to the GPU by using the function `.to(...)`, or `.cuda()`. However, it is often a good practice to define a `device` object in your code which points to the GPU if you have one, and otherwise to the CPU. Then, you can write your code with respect to this device object, and it allows you to run the same code on both a CPU-only system, and one with a GPU. Let's try it below. We can specify the device as follows: 

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Let's create a large tensor, push it to device.

In [ ]:
x = torch.randn(1000, 1000, 1000).to(device) 
x.dtype, x.device

In [ ]:
create_answer_box(
    "PREDICTION: We just created a tensor of size (1000, 1000, 1000). How much GPU memory do you think this will use?", 
    "02-02"
)

In [ ]:
!nvidia-smi

In [ ]:
del x

# the gpu memory may not be freed right away
# we can explicitly free
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

**Tensors must be on the same device**

In [ ]:
a = torch.randn(3,3, device = torch.device('cuda')) # GPU
b = torch.randn(3,3) # default

try:
    print(a+b)
except RuntimeError as e:
    print(e)

**This will happen to you a lot**

In [ ]:
try:
    # tensor is too big to fit on our GPU!
    x = torch.randn(int(3e10), device=torch.device('cuda'))
except Exception as e:
    print(e)

The solution: find ways to use less memory.

**How much does GPU actually help?**

Let's test by multiplying a large matrix with itself. 

Before running the below code cell: we are multiplying two 10,000 x 10,000 matrices on CPU vs GPU. How much faster do you think the GPU will be? 

In [ ]:
import time

x = torch.randn(10_000, 10_000)

## CPU version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
cpu_time = end_time - start_time
print(f"CPU time: {(cpu_time):6.5f}s")

## GPU version
x = x.to(device)
_ = torch.matmul(x, x)  # First operation to 'burn in' GPU
# CUDA is asynchronous, so we need to use different timing functions
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
_ = torch.matmul(x, x)
end.record()
torch.cuda.synchronize()  # Waits for everything to finish running on the GPU
gpu_time = start.elapsed_time(end) / 1000  # Milliseconds to seconds
print(f"GPU time: {gpu_time:6.5f}s")  # Milliseconds to seconds

# How much faster is the GPU?
print(f"The GPU is {cpu_time / gpu_time:6.1f}x faster than the CPU")

## Computation Graph and Backpropagation

One of the main reasons for using PyTorch in Deep Learning projects is that we can automatically get _gradients/derivatives_ of functions that we define. We will mainly use PyTorch for implementing neural networks, and they are just fancy functions. If we use weight matrices in our function that we want to learn, then those are called the _parameters_ or simply the _weights_. The ability to compute gradients is essential for optimizing (a.k.a. _training_) our networks. 

Tensors have a `requires_grad` attribute

In [ ]:
x = torch.ones((3,))
print(x.requires_grad)

We can change this for an existing tensor using the function `requires_grad_()` (underscore indicating that this is a in-place operation). Alternatively, when creating a tensor, you can pass the argument `requires_grad=True` to most initializers we have seen above.

In [ ]:
x.requires_grad_(True)
print(x.requires_grad)

In order to get familiar with the concept of a computation graph, we will create one for the following function:

$$y = \frac{1}{\mathrm{dim}(x)}\sum_i \left[(x_i + 2)^2 + 3\right]$$

You could imagine that $x$ are our parameters, and we want to optimize (either maximize or minimize) the output $y$. For this, we want to obtain the gradients $\partial y / \partial \mathbf{x}$. For our example, we'll use $\mathbf{x}=[0,1,2]$ as our input.

In [ ]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True) # Only float tensors can have gradients
print("X", x)

Now let's build the computation graph step by step. You can combine multiple operations in a single line, but we will separate them here to get a better understanding of how each operation is added to the computation graph.

In [ ]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()
print("Y", y)

Using the statements above, we have created a computation graph that looks similar to the figure below:

<center style="width: 100%"><img src="https://raw.githubusercontent.com/clemsonciti/rcde_workshops/master/pytorch/forward_computation.svg" width="200px"></center>

We calculate $a$ based on the inputs $x$ and the constant $2$, $b$ is $a$ squared, and so on. The visualization is an abstraction of the dependencies between inputs and outputs of the operations we have applied.

Each node in the computation graph has automatically defined a function for calculating the gradients with respect to its inputs, `grad_fn`. Pytorch can use the chain rule to automatically compute the gradients of $y$ with respect to any of the inputs that have `requires_grad=True`. Pytorch does this by traversing the computation graph backward, applying the `grad_fn` defined at each operation. This is called "backpropagation". 

<center style="width: 100%"><img src="https://raw.githubusercontent.com/clemsonciti/rcde_workshops/master/pytorch/backward_computation.svg" width="200px"></center>


We can perform backpropagation on the computation graph by calling the function `backward()` on the last output:

In [ ]:
y.backward()

`x.grad` will now contain the gradient $\partial y/ \partial \mathcal{x}$, and this gradient indicates how a change in $\mathbf{x}$ will affect output $y$ given the current input $\mathbf{x}=[0,1,2]$:

In [ ]:
print(x.grad)

We can also verify these gradients by hand. We will calculate the gradients using the chain rule, in the same way as PyTorch did it:

$$
\frac{\partial y}{\partial x_i} = \frac{\partial y}{\partial c_i}\frac{\partial c_i}{\partial b_i}\frac{\partial b_i}{\partial a_i}\frac{\partial a_i}{\partial x_i}
$$

Note that we have simplified this equation to index notation, and by using the fact that all operation besides the mean do not combine the elements in the tensor. The partial derivatives are:

$$
\frac{\partial a_i}{\partial x_i} = 1,\hspace{1cm}
\frac{\partial b_i}{\partial a_i} = 2\cdot a_i\hspace{1cm}
\frac{\partial c_i}{\partial b_i} = 1\hspace{1cm}
\frac{\partial y}{\partial c_i} = \frac{1}{3}
$$

Hence, with the input being $\mathbf{x}=[0,1,2]$, our gradients are $\partial y/\partial \mathbf{x}=[4/3,2,8/3]$. The previous code cell should have printed the same result.

In [ ]:
# CODE CHALLENGE: Use pytorch to find the gradient of the multivariate function f(x, y) = x^2 + 2y^2 at the point (x, y) = (1, 2).


In [ ]:
create_answer_box("Copy/paste your code to find the gradient of f(x, y) = x^2 + 2y^2 at (1, 2)", "02-03")